### Classification metrics

In the last notebook, you fit a binary classifier to predict whether patients are diabetic. You used accuracy as a measure of how well the model performed, but accuracy isn't everything. In this exercise, we'll look at alternatives to accuracy that can be much more useful in machine learning.

### Alternative metrics for binary classifiers

Accuracy seems like a sensible metric for evaluating a model, and to a certain extent it is, but you need to be careful about drawing too many conclusions from the accuracy of a classifier. Remember that it's simply a measure of how many cases were predicted correctly. Let's suppose that only 3 percent of the population is diabetic. You could create a classifier that always predicts 0, and it would be 97 percent accurate, but not terribly helpful in identifying patients with diabetes.

Fortunately, there are some other metrics that reveal a little more about how our classification model is performing.


In [ ]:
# Load the required packages and make them available in your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(janitor)
  library(tidymodels)
})


# Read the CSV file into a tibble
diabetes <- read_csv(file = "https://raw.githubusercontent.com/MicrosoftDocs/ml-basics/master/data/diabetes.csv", show_col_types = FALSE)


# Clean data a bit
diabetes_select <- diabetes %>%
  # Encode Diabetic as category
  mutate(Diabetic = factor(Diabetic, levels = c("1","0"))) %>% 
  # Drop PatientID column
  select(-PatientID) %>% 
  # Clean column names
  clean_names()


# Split data into 70% for training and 30% for testing
set.seed(2056)
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.70)


# Extract the data in each split
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)


# Make a model specification
logreg_spec <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


# Train a logistic regression model
logreg_fit <- logreg_spec %>% 
  fit(diabetic ~ ., data = diabetes_train)


# Make predictions and then bind them to the test set
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(logreg_fit %>% predict(new_data = diabetes_test))


# Calculate accuracy: proportion of data predicted correctly
accuracy(data = results, truth = diabetic, estimate = .pred_class)


One performance metric that's associated with classification problems is the [confusion matrix](https://wikipedia.org/wiki/Confusion_matrix). A confusion matrix describes how well a classification model performs by tabulating how many examples in each class were correctly classified by a model. In our scenario, the matrix shows you how many cases were classified as negative (0) and how many as positive (1). It also shows you how many cases were classified into the *wrong* categories.

The [`conf_mat()`](https://tidymodels.github.io/yardstick/reference/conf_mat.html) function from yardstick calculates this cross-tabulation of observed and predicted classes.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = diabetic, estimate = .pred_class)


Now, let's interpret the confusion matrix. Your model is asked to classify cases between two binary categories, category *1* for patients who tested positive for diabetes and category *0* for patients who tested negative.

- If your model predicts a patient as *1* (positive) and they belong to category *1* (positive) in reality, we call this a *true positive* (TP), as shown by the top left number *897*.

- If your model predicts a patient as *0* (negative) and they belong to category *1* (positive) in reality, we call this a *false negative* (FN), as shown by the bottom left number *657*.

- If your model predicts a patient as *1* (positive) and they belong to category *0* (negative) in reality, we call this a *false positive* (FP), as shown by the top right number *293*.

- If your model predicts a patient as *0* (negative) and they belong to category *0* (negative) in reality, we call this a *true negative* (TN), as shown by the bottom right number *2653*.

Your confusion matrix can thus be expressed in the following form:

| Truth |
|:-----:|

|               |                  |                   |
|:-------------:|:----------------:|:-----------------:|
| **Predicted** |        1         |         0         |
|       1       | $897_{\ \ \ TP}$ | $293_{\ \ \ FP}$  |
|       0       | $657_{\ \ \ FN}$ | $2653_{\ \ \ TN}$ |

Note that the correct (*true*) predictions form a diagonal line from top left to bottom right. If the model is any good, these numbers should be significantly greater than the *false* predictions.

The confusion matrix is helpful because it gives rise to other metrics that can help you better evaluate the performance of a classification model. Let's go through some of them:

* **Precision**: *TP/(TP + FP)*, the proportion of predicted positives that are actually positive (also called [positive predictive value](https://en.wikipedia.org/wiki/Positive_predictive_value)).

* **Recall**: *TP/(TP + FN)*, the proportion of positive results out of the number of samples that were actually positive (also known as *sensitivity*).

* **Specificity**: *TN/(TN + FP)*, the proportion of negative results out of the number of samples that were actually negative.

* **Accuracy**: *TP + TN/(TP + TN + FP + FN)*, the proportion of labels that were predicted accurately for a sample.

* **F Measure**: The weighted average of the precision and recall metrics, with 1 as the best and 0 as the worst.

Tidymodels provides yet another succinct way to evaluate all these metrics. By using `yardstick::metric_set()`, you can combine multiple metrics into a new function that calculates all of them at once.


In [ ]:
# Combine metrics and evaluate them all at once
eval_metrics <- metric_set(ppv, recall, accuracy, f_meas)
eval_metrics(data = results, truth = diabetic, estimate = .pred_class)


By using the precision (ppv) metric, you can now answer the question:

- Of all the patients the model predicted are diabetic, how many are actually diabetic?

By using the recall metric, you can now answer the question:

- Of all the patients that are actually diabetic, how many did the model identify?

Great job! You've just made predictions and evaluated them by using a variety of metrics.

Until now, you've considered the predictions from the model either 1 or 0 class labels. Actually, things are a little more complex than that. Statistical machine-learning algorithms, such as logistic regression, are based on *probability*, so what the binary classifier actually predicts is both the probability that the label is true ($P(y)$) and the probability that the label is false ($1-P(y)$). Let's use a threshold value of 0.5 to decide whether the predicted label is a 1 ($P(y)>0.5$) or a 0 ($P(y)<=0.5$). Now, predict the probability pairs for each case:


In [ ]:
# Predict class probabilities and bind them to results
results <- results %>% 
  bind_cols(logreg_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print out the results
results %>% 
  slice_head(n = 10)


The decision to score a prediction as a 1 or a 0 depends on the threshold to which the predicted probabilities are compared. If you were to change the threshold, it would affect the predictions and, therefore, change the metrics in the confusion matrix. 

A common way to evaluate a classifier is to examine the *true positive rate* (which is another name for recall) and the *false positive rate* (1 - specificity) for a range of possible thresholds. These rates are then plotted against all possible thresholds to form a chart known as a *received operator characteristic (ROC) chart*, as shown here:



In [ ]:
# Make a roc_chart
results %>% 
  roc_curve(truth = diabetic, .pred_1) %>% 
  autoplot()


The ROC chart shows the curve of the true and false positive rates for different threshold values from 0 to 1. A perfect classifier would have a curve that goes straight up the left side and straight across the top. The diagonal line across the chart represents the probability of predicting correctly with a 50/50 random prediction. So you obviously want the curve to be higher than that (or your model is no better than a simple guess).

The area under the curve (AUC) is a value from 0 to 1 that quantifies the overall performance of the model. One way to interpret AUC is as the probability that the model ranks a random positive example more highly than a random negative example. The closer to 1 this value is, the better the model. Once again, tidymodels includes a function to calculate this metric: `yardstick::roc_auc()`


In [ ]:
# Compute the AUC
results %>% 
  roc_auc(diabetic, .pred_1)


### Data preprocessing with recipes

In this case, the ROC curve and its AUC indicate that the model performs better than a random guess, which is not bad considering that you performed very little preprocessing of the data.

In practice, it's common to perform some preprocessing of the data to make it easier for the algorithm to fit a model to it. There's a huge range of preprocessing transformations you can perform to get your data ready for modeling, but we'll limit ourselves to a few common techniques:

- **Scaling numeric features so they're on the same scale**: This prevents features with large values from producing coefficients that disproportionately affect the predictions.
- **Encoding categorical variables**: For example, by using a *one hot encoding* technique, you can create "*dummy*" or *indicator variables* that replace the original categorical feature with numeric columns whose values are either 1 or 0.

Tidymodels provides yet another neat package: [recipes](https://recipes.tidymodels.org/), a package for preprocessing data. As an example, let's specify a recipe that encodes the age column and then normalizes the rest of the predictor features:


In [ ]:
# Preprocess the data for modeling
diabetes_recipe <- recipe(diabetic ~ ., data = diabetes_train) %>% 
  step_mutate(age = factor(age)) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(age)

# Print the recipe
diabetes_recipe


You've just created a recipe containing an outcome and its corresponding predictors. In doing so, you've specified that the age variable should be converted to a categorical variable (factor), normalized all the numeric predictors, and created dummy variables for the nominal predictor (age).

### Bundle it all by using a workflow

Now that you have a recipe and a model specification that you defined previously, you need to find a way to bundle them into an object that first preprocesses the data, and then both fit the model on the preprocessed data and allow for potential post-processing activities.

In tidymodels, this convenient object is called a [workflow](https://workflows.tidymodels.org/), and it conveniently holds your modeling components.

With the `workflows` package, you can bind the modeling and preprocessing objects together. You can then fit the entire workflow to the data, so that the model encapsulates all the preprocessing steps and the algorithm, as shown here:


In [ ]:
# Redefine the model specification
logreg_spec <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")

# Bundle the recipe and model specification
lr_wf <- workflow() %>% 
  add_recipe(diabetes_recipe) %>% 
  add_model(logreg_spec)

# Print the workflow
lr_wf


After you've specified a workflow, you can train a model by using the [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html) function, as shown here:



In [ ]:
# Fit a workflow object
lr_wf_fit <- lr_wf %>% 
  fit(data = diabetes_train)

# Print wf object
lr_wf_fit


Good job! You now have a trained workflow. The workflow's printout shows the coefficients that were learned during training.

You can now use the model trained by this workflow to predict labels for your test set, and compare the performance metrics with the basic model that you created previously.


In [ ]:
# Make predictions on the test set
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(lr_wf_fit %>% 
              predict(new_data = diabetes_test)) %>% 
  bind_cols(lr_wf_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print the results
results %>% 
  slice_head(n = 10)


Let's take a look at the confusion matrix:



In [ ]:
# Confusion matrix for prediction results
results %>% 
  conf_mat(truth = diabetic, estimate = .pred_class)


Look at those metrics!

Can you visualize these proportions? Of course, nothing is "impaRsible"!


In [ ]:
# Visualize conf mat
update_geom_defaults(geom = "rect", new = list(fill = "midnightblue", alpha = 0.7))

results %>% 
  conf_mat(diabetic, .pred_class) %>% 
  autoplot()


What about other performance metrics that you might derive from the confusion matrix?



In [ ]:
# Evaluate other desired metrics
eval_metrics(data = results, truth = diabetic, estimate = .pred_class)


Finally, let's take a look at the ROC metrics.



In [ ]:
# Evaluate ROC_AUC metrics
results %>% 
  roc_auc(diabetic, .pred_1)

# Plot ROC_CURVE
results %>% 
  roc_curve(diabetic, .pred_1) %>% 
  autoplot()


As compared with previous predictions, these metrics look better. Clearly, preprocessing the data has made a difference!

### Try a different algorithm

Now let's try a different algorithm. Earlier, you used a logistic regression algorithm, which is *linear*. There are many kinds of classification algorithms you might try, including:

- **Support vector machine algorithm**: Defines a *hyperplane* that separates classes.
- **Tree-based algorithm**: Builds a decision tree to reach a prediction.
- **Ensemble algorithm**: Combines the outputs of multiple base algorithms to improve generalizability.

This time, let's train the model by using an *ensemble* algorithm called *random forest*, which averages the outputs of multiple random decision trees. Random forests help to reduce tree correlation by injecting more randomness into the tree-growing process. More specifically, instead of considering all predictors in the data, for calculating a given split, random forests pick a random sample of predictors to be considered for that split.

You've already got the gist of how to perform classification by using tidymodels, so let's get right into specifying and fitting a random forest algorithm.


In [ ]:
# Preprocess the data for modeling
diabetes_recipe <- recipe(diabetic ~ ., data = diabetes_train) %>% 
  step_mutate(age = factor(age)) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  step_dummy(age)

# Build a random forest model specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger", importance = "impurity") %>% 
  set_mode("classification")

# Bundle recipe and model spec into a workflow
rf_wf <- workflow() %>% 
  add_recipe(diabetes_recipe) %>% 
  add_model(rf_spec)

# Fit a model
rf_wf_fit <- rf_wf %>% 
  fit(data = diabetes_train)

# Make predictions on test data
results <- diabetes_test %>% select(diabetic) %>% 
  bind_cols(rf_wf_fit %>% 
              predict(new_data = diabetes_test)) %>% 
  bind_cols(rf_wf_fit %>% 
              predict(new_data = diabetes_test, type = "prob"))

# Print out predictions
results %>% 
  slice_head(n = 10)


There goes our random_forest model. Is it any good? Let's evaluate its metrics!



In [ ]:
# Confusion metrics for rf_predictions
results %>% 
  conf_mat(diabetic, .pred_class)

# Confusion matrix plot
results %>% 
  conf_mat(diabetic, .pred_class) %>% 
  autoplot()


There's a considerable increase in the number of *true positives* and *true negatives*, which is a step in the right direction.

Let's take a look at other evaluation metrics.


In [ ]:
# Evaluate other intuitive classification metrics
rf_met <- results %>% 
  eval_metrics(truth = diabetic, estimate = .pred_class)

# Evaluate ROC_AUC
auc <- results %>% 
  roc_auc(diabetic, .pred_1)

# Plot ROC_CURVE
curve <- results %>% 
  roc_curve(diabetic, .pred_1) %>% 
  autoplot

# Return metrics
list(cm_metrics = rf_met, AUC = auc, ROC_CURVE = curve)


For the sheer sake of adventure, let's make a *variable importance plot* (VIP) to see which predictor variables have the most impact in our model.



In [ ]:
# Load vip
library(vip)

# Extract the fitted model from the workflow
rf_wf_fit %>% 
  extract_fit_parsnip() %>% 
# Make VIP plot
  vip()


The VIP curve just reinforced what we put forward during data exploration. This goes to show the importance of data exploration.

As revealed by the performance metrics, the random forest model seemed to have done a great job in increasing the true positives and negatives and decreasing the false positives and negatives.

### Use the model for inferencing

Now that you have a reasonably useful trained model, you can save it for later use to predict labels for new data:


In [ ]:
library(here)
# Save trained workflow in the location of this script
saveRDS(rf_wf_fit, "diabetes_rf_model.rds")


You can load it whenever you need it, and then use it to predict labels for new data. This is often called *scoring* or *inferencing*.

For example, how about creating a simulated dataset by picking a random value for each column in your test set and then making predictions based on the saved model.


In [ ]:
# Load the model into the current R session
loaded_model <- readRDS("diabetes_rf_model.rds")

# Create new simulated data
new_data <- map_dfc(diabetes_test, function(x){sample(x, size = 2)}) %>% 
  as_tibble()

# Make predictions
predictions <- new_data %>% 
  bind_cols(loaded_model %>% predict(new_data))

predictions


### Summary

In this exercise, you looked at various metrics to evaluate a binary classification model. This exploration gave you further information about the accuracy of the model's performance on new, unlabeled data. You trained and evaluated two classification models, a logistic regression model and a random forest model, against the patient dataset, and you compared the performance outcomes of the two. In the next unit, you'll deal with multi-class classification models.